In [1]:
import os
import sys
import gensim
import pandas as pd
import pickle
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec
from sklearn.cross_validation import train_test_split
from gensim.models.word2vec import Word2Vec
from tqdm import tqdm
from sklearn import utils
import numpy as np
from keras import optimizers
from keras.models import load_model
from sklearn.metrics import accuracy_score, classification_report
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, GRU, Bidirectional

C:\Users\User\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\User\Anaconda3\lib\site-packages\pattern-2.6-py3.6.egg\pattern\text\en\..\..\..\..\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\User\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using 

In [2]:
# train_set = pd.read_csv('./corpus/tripadvisor/train_set.csv')
test_set = pd.read_csv('./corpus/tripadvisor/test.csv')
test_set.head()

,content,polarity
0,lokasi hotel tidak jauh dari komplek mall kali...,negative
1,lokasi hotel tidak jauh dengan taksi ke area m...,negative
2,lokasi hotel yang sangat strategis masih di ko...,negative
3,lokasi hotel yang sangat strategis masih di ko...,negative
4,lokasi hotel yang strategis di tengah kota dan...,negative


In [3]:
def sentiment_label(polarity):
    if polarity=='negative':
        return 0
    else:
        return 1

In [4]:
# train_set['sentiment'] = train_set['polarity'].apply(sentiment_label)
test_set['sentiment'] = test_set['polarity'].apply(sentiment_label)
test_set.head()

,content,polarity,sentiment
0,lokasi hotel tidak jauh dari komplek mall kali...,negative,0
1,lokasi hotel tidak jauh dengan taksi ke area m...,negative,0
2,lokasi hotel yang sangat strategis masih di ko...,negative,0
3,lokasi hotel yang sangat strategis masih di ko...,negative,0
4,lokasi hotel yang strategis di tengah kota dan...,negative,0


In [5]:
SEED = 2000

x_train, x_validation, y_train, y_validation = train_test_split(test_set['content'], test_set['sentiment'], test_size=.1, random_state=SEED)

In [6]:
# x_train = train_set['content']
# x_validation = test_set['content']
# y_train = train_set['sentiment']
# y_validation = test_set['sentiment']

In [7]:
def labelize_text(text,label):
    result = []
    prefix = label
    for i, t in zip(text.index, text):
        result.append(LabeledSentence(t.split(), [prefix + '_%s' % i]))
    return result
  
x_train = labelize_text(x_train, 'TRAIN')
x_validation = labelize_text(x_validation, 'TEST')

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  """


In [8]:
MAX_SEQUENCE_LENGTH = 85
data_dim = 700

In [9]:
# word2vec = Word2Vec.load("./vectorizer/tripadvisor/word2vec_300.model")
word2vec = Word2Vec.load('./prosa-w2v/prosa.vec')
tfidf = pickle.load(open('./vectorizer/tripadvisor/tfidf.pickle', 'rb'))
model_dbow = Doc2Vec.load("./vectorizer/tripadvisor/model_dbow.model")
model_dmc = Doc2Vec.load("./vectorizer/tripadvisor/model_dmc.model")
model_dmm = Doc2Vec.load("./vectorizer/tripadvisor/model_dmm.model")

def build_doc_Vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += np.append(model_dbow[word] * tfidf[word], model_dmm[word] * tfidf[word])
            count += 1
        except KeyError: 
            continue
    if count != 0:
        vec /= count
    return vec

def build_Vector(tokens, word_size, doc_size):
    doc_vec = build_doc_Vector(tokens, doc_size)
    vec = np.zeros((MAX_SEQUENCE_LENGTH - len(tokens), doc_size + word_size))
    for word in tokens:
        try:
            word_vec = np.append(doc_vec, word2vec[word])
            vec = np.append(vec, word_vec)
        except KeyError: 
            word_vec = np.append(doc_vec, np.zeros((1, word_size)))
            vec = np.append(vec, word_vec)
            continue
    vec.reshape(MAX_SEQUENCE_LENGTH, doc_size + word_size)
    return vec

In [10]:
train_vecs = np.concatenate([[build_Vector(z, 500, 200)] for z in tqdm(map(lambda x: x.words, x_train))])
val_vecs = np.concatenate([[build_Vector(z, 500, 200)] for z in tqdm(map(lambda x: x.words, x_validation))])

0it [00:00, ?it/s]C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
746it [00:03, 194.02it/s]
83it [00:00, 194.51it/s]


In [11]:
batch_size = 56
num_epochs = 10
hidden_size = 10
timesteps = MAX_SEQUENCE_LENGTH
num_class = 1

In [12]:
num_data = len(train_vecs)
num_data_val = len(val_vecs)

train_vecs = train_vecs.reshape((num_data, timesteps, data_dim))
y_train = y_train.reshape((num_data, num_class))
val_vecs = val_vecs.reshape((num_data_val, timesteps, data_dim))
y_validation = y_validation.reshape((num_data_val, num_class))

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  import sys


In [13]:
model = Sequential()
model.add(Bidirectional(GRU(hidden_size, input_shape=(timesteps, data_dim)), merge_mode='concat'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_vecs, y_train, epochs=num_epochs, validation_data=[val_vecs, y_validation])

Train on 746 samples, validate on 83 samples
Epoch 1/10
746/746 [==============================] - ETA: 1:32 - loss: 0.7241 - acc: 0.625 - ETA: 46s - loss: 0.7200 - acc: 0.531 - ETA: 31s - loss: 0.7704 - acc: 0.47 - ETA: 23s - loss: 0.7470 - acc: 0.53 - ETA: 18s - loss: 0.7413 - acc: 0.54 - ETA: 15s - loss: 0.7140 - acc: 0.56 - ETA: 12s - loss: 0.7063 - acc: 0.58 - ETA: 10s - loss: 0.6984 - acc: 0.59 - ETA: 9s - loss: 0.6961 - acc: 0.5868 - ETA: 8s - loss: 0.6897 - acc: 0.596 - ETA: 7s - loss: 0.6808 - acc: 0.613 - ETA: 6s - loss: 0.6650 - acc: 0.625 - ETA: 5s - loss: 0.6651 - acc: 0.617 - ETA: 4s - loss: 0.6601 - acc: 0.622 - ETA: 3s - loss: 0.6482 - acc: 0.637 - ETA: 3s - loss: 0.6520 - acc: 0.638 - ETA: 2s - loss: 0.6455 - acc: 0.648 - ETA: 2s - loss: 0.6385 - acc: 0.654 - ETA: 1s - loss: 0.6312 - acc: 0.659 - ETA: 1s - loss: 0.6267 - acc: 0.662 - ETA: 0s - loss: 0.6192 - acc: 0.671 - ETA: 0s - loss: 0.6150 - acc: 0.673 - ETA: 0s - loss: 0.6085 - acc: 0.678 - 10s 13ms/step - loss: 0

746/746 [==============================] - ETA: 4s - loss: 0.1622 - acc: 0.968 - ETA: 4s - loss: 0.1763 - acc: 0.953 - ETA: 4s - loss: 0.1568 - acc: 0.958 - ETA: 3s - loss: 0.1359 - acc: 0.968 - ETA: 3s - loss: 0.1290 - acc: 0.975 - ETA: 3s - loss: 0.1254 - acc: 0.968 - ETA: 3s - loss: 0.1150 - acc: 0.973 - ETA: 3s - loss: 0.1217 - acc: 0.968 - ETA: 2s - loss: 0.1152 - acc: 0.972 - ETA: 2s - loss: 0.1118 - acc: 0.975 - ETA: 2s - loss: 0.1225 - acc: 0.968 - ETA: 2s - loss: 0.1183 - acc: 0.971 - ETA: 2s - loss: 0.1129 - acc: 0.973 - ETA: 1s - loss: 0.1145 - acc: 0.973 - ETA: 1s - loss: 0.1126 - acc: 0.972 - ETA: 1s - loss: 0.1136 - acc: 0.970 - ETA: 1s - loss: 0.1123 - acc: 0.972 - ETA: 1s - loss: 0.1163 - acc: 0.972 - ETA: 0s - loss: 0.1258 - acc: 0.968 - ETA: 0s - loss: 0.1217 - acc: 0.970 - ETA: 0s - loss: 0.1208 - acc: 0.971 - ETA: 0s - loss: 0.1186 - acc: 0.973 - ETA: 0s - loss: 0.1169 - acc: 0.972 - 5s 7ms/step - loss: 0.1159 - acc: 0.9732 - val_loss: 0.1531 - val_acc: 0.9639
Epoch

In [14]:
# model.save('./model/bi_gru_pv/bi_gru_model_01.h5')  

In [15]:
# model = load_model('./model/bi_gru_pv/bi_gru_model_01.h5')
y_pred = model.predict(val_vecs)
for i in range(len(y_pred)):
    y_pred[i][0] = round(y_pred[i][0])

print("Accuracy: ", accuracy_score(y_validation, y_pred))
print(classification_report(y_validation, y_pred, labels = [0, 1], digits=8))

Accuracy:  0.963855421686747
             precision    recall  f1-score   support

          0  0.96428571 0.98181818 0.97297297        55
          1  0.96296296 0.92857143 0.94545455        28

avg / total  0.96383948 0.96385542 0.96368965        83

